In [2]:
import pandas as pd
import numpy as np

SURVEY = pd.read_csv('lv2-2403.csv')
print(SURVEY.shape)
SURVEY.info()

(11351, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11351 entries, 0 to 11350
Data columns (total 11 columns):
D_ID          11351 non-null int64
HOBBY         11351 non-null int64
OPENSOURCE    11351 non-null int64
CORPORATE     11351 non-null int64
YEARS         11351 non-null object
SATIS         11351 non-null float64
SALARY        8526 non-null float64
MONITORS      11351 non-null int64
TOOL          11351 non-null object
GENDER        9705 non-null object
AGE           11351 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 975.6+ KB


### Question 1

In [3]:
# 1-1
df1 = SURVEY[SURVEY['SALARY'].isnull()==False].copy()

In [4]:
# 1-2
AVG1 = df1['SALARY'].mean()
AVG1

98525.35186488388

In [5]:
# 1-3
df1['m'] = df1.groupby('YEARS').SALARY.transform('mean')
df1['s'] = df1.groupby('YEARS').SALARY.transform('std')
df1['out'] = 1*(df1['SALARY']<(df1['m']-1.5*df1['s'])) + 1*(df1['SALARY']>(df1['m']+1.5*df1['s'])) 
df1_2 = df1[df1['out']==0].copy()
print(sum(df1['out']==1))
print(df1_2.shape[0])

312
8214


In [6]:
# 1-4
AVG2 = df1_2['SALARY'].mean()
AVG2

64055.169345020695

In [7]:
# Answer 1
print(abs(AVG1-AVG2))
ans21 = round(abs(AVG1-AVG2)).astype('int')
ans21

34470.182519863185


34470

### Question 2

In [8]:
# 2-1
DF2 = SURVEY[SURVEY['AGE']=='25-34 years old'].copy().reset_index()
DF2.shape

(6209, 12)

In [9]:
# 2-2
DF2['SALARY'] = DF2.groupby('YEARS')['SALARY'].apply(lambda x: x.fillna(x.median()))
DF2 = DF2.sort_values(['D_ID'])

In [10]:
# 2-3
DF2['TOOL_CNT'] = DF2['TOOL'].str.count(';') + 1

In [11]:
# 2-4
from sklearn.preprocessing import StandardScaler
SVAR = ['SATIS', 'SALARY', 'MONITORS', 'TOOL_CNT']
SVAR_S = ['SATIS_S', 'SALARY_S', 'MONITORS_S', 'TOOL_CNT_S']
scaler = StandardScaler()
scaled = pd.DataFrame(scaler.fit_transform(DF2[SVAR]), columns = SVAR_S)
DF2 = pd.concat([DF2.drop(columns=SVAR), scaled], axis=1)
DF2.columns

Index(['index', 'D_ID', 'HOBBY', 'OPENSOURCE', 'CORPORATE', 'YEARS', 'TOOL',
       'GENDER', 'AGE', 'SATIS_S', 'SALARY_S', 'MONITORS_S', 'TOOL_CNT_S'],
      dtype='object')

In [12]:
# 2-5
from sklearn.cluster import KMeans 
DF2['clst'] = KMeans(n_clusters=2, random_state=1234).fit_predict(DF2[SVAR_S])

In [13]:
# 2-6
GRATIO = DF2.groupby('clst').apply(lambda x: (x['GENDER']=='Male').sum()/(x['GENDER']=='Female').sum()*100).reset_index(name='G')
GRATIO

,clst,G
0,0,188.310709
1,1,181.767956


In [14]:
# Answer 2
print(GRATIO.G.max()-GRATIO.G.min())
ans22 = round(GRATIO.G.max()-GRATIO.G.min(), 2)
print(ans22)

6.542753097839238
6.54


### Question 3

In [15]:
# 3-1
df3 = SURVEY.copy()
df3 = df3[df3['GENDER'].isnull()==False]
df3 = df3[df3['SALARY'].isnull()==False]
print(df3.shape)

(7431, 11)


In [16]:
# 3-2
df3['GIT_YN']=df3['TOOL'].apply(lambda x : 1 if 'Git' in x else 0 )

In [17]:
# 3-3
dvar3 = ['YEARS', 'GENDER','AGE']
dummy3 = pd.get_dummies(df3[dvar3], drop_first=False)
df3d = pd.concat([df3,dummy3], axis=1)
df3d.drop(['YEARS','TOOL','GENDER','AGE'], axis=1, inplace=True)

In [18]:
# 3-4
trainset3 = df3d[df3d['D_ID']%3 !=0]
testset3 = df3d[df3d['D_ID']%3==0]
print(trainset3.shape, testset3.shape)

(4917, 24) (2514, 24)


In [19]:
# 3-5
from sklearn.tree import DecisionTreeRegressor
COL3 = ['HOBBY','OPENSOURCE','CORPORATE','SALARY','MONITORS','GIT_YN'] + dummy3.columns.tolist()
MDL3 = DecisionTreeRegressor(max_depth=7, random_state=1234).fit(trainset3[COL3], trainset3['SATIS'])

In [20]:
# 3-6
PRED3 = MDL3.predict(testset3[COL3])
MAPE = np.mean(np.abs((PRED3 - testset3.SATIS)/testset3.SATIS))*100
MAPE

51.33212768378126

In [21]:
# Answer 3
ans23 = round(MAPE, 1)
ans23

51.3

## Summary

In [22]:
print('Q1: ', ans21)
print('Q2: ', ans22)
print('Q3: ', ans23)

Q1:  34470
Q2:  6.54
Q3:  51.3
